In [1]:
import torch
from transformers import (AutoTokenizer, AutoModelForQuestionAnswering)
from datasets import QA_Dataset
from torch.utils.data import DataLoader

from utils import same_seed, read_data

device =  'cuda'
print(device)

c:\Users\Ryan\anaconda3\envs\mlhw07\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
model_save_dir = "./saved_model/" 

In [3]:
# seeding
same_seed(5201314)

model_save_dir = "./saved_model/" 

model = AutoModelForQuestionAnswering.from_pretrained(model_save_dir).to(device)
model_name = "luhua/chinese_pretrain_mrc_macbert_large"
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [4]:
# test
test_questions, test_paragraphs = read_data("data/hw7_test.json")
test_questions_tokenized = tokenizer([test_question['question_text'] for test_question in test_questions], add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


In [5]:
doc_stride = 50

def evaluate(data, output, tokenizer, idx):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            pre_answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

        # if '[UNK]' in pre_answer:
            start_index += doc_stride * k - len(test_questions_tokenized[idx]) - 2
            end_index += doc_stride * k - len(test_questions_tokenized[idx]) - 2
            if start_index < 0 or end_index < 0:
                continue
            try:
                tokenized_paragraph = test_paragraphs_tokenized[test_questions[idx]['paragraph_id']]
                start_index = min(tokenized_paragraph.token_to_chars(start_index))
                end_index = max(tokenized_paragraph.token_to_chars(end_index))
            except:
                print(f"error! :{start_index}, {end_index}")
                continue
            # print(test_paragraphs[test_questions[idx]['paragraph_id']][start_index:end_index])
            pre_answer = test_paragraphs[test_questions[idx]['paragraph_id']][start_index:end_index]
            if pre_answer != "" or len(pre_answer) < 15:
                answer = pre_answer
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    answer = answer.replace(' ','')

    if answer == "":
        print("empty~ num~ unlls")
    return answer

In [6]:
test_set[0]

(tensor([[ 101, 8271, 2399,  ...,    0,    0,    0],
         [ 101, 8271, 2399,  ...,    0,    0,    0],
         [ 101, 8271, 2399,  ...,    0,    0,    0],
         ...,
         [ 101, 8271, 2399,  ...,    0,    0,    0],
         [ 101, 8271, 2399,  ...,    0,    0,    0],
         [ 101, 8271, 2399,  ...,    0,    0,    0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]))

In [7]:
print(test_questions[0]['question_text'])
print(len(test_questions[0]['question_text']))

2018年世界盃外圍賽中，韓國打敗兩個國家的代表隊，取得2018年世界盃決賽周參賽資格，其中一個國家是敘利亞，另一個國家是?
62


In [8]:

token = tokenizer(test_questions[0]['question_text'], add_special_tokens=False)
print(token)
print(len(token['input_ids']))

{'input_ids': [8271, 2399, 686, 4518, 4656, 1912, 1752, 6555, 704, 8024, 7502, 1751, 2802, 3134, 1060, 943, 1751, 2157, 4638, 807, 6134, 7386, 8024, 1357, 2533, 8271, 2399, 686, 4518, 4656, 3748, 6555, 1453, 1347, 6555, 6536, 3419, 8024, 1071, 704, 671, 943, 1751, 2157, 3221, 3135, 1164, 765, 8024, 1369, 671, 943, 1751, 2157, 3221, 136], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
56


In [9]:
test_paragraphs[test_questions[0]['paragraph_id']]

'2014年世界盃外圍賽，韓國在首輪分組賽以首名出線次輪分組賽，與伊朗、卡達、烏茲別克以及黎巴嫩爭逐兩個直接出線決賽周資格，最後韓國僅以較佳的得失球差壓倒烏茲別克，以小組次名取得2014年世界盃決賽周參賽資格，也是韓國連續八次晉身世界盃決賽周。可惜南韓在決賽周表現不濟，三戰一和兩負小組末席出局。2018年世界盃外圍賽，韓國再次在首輪分組賽以首名出線次輪分組賽，再與伊朗、卡達、烏茲別克同組，同組還有中國及敘利亞。最後韓國以兩分壓倒敘利亞及烏茲別克，再以小組次名取得2018年世界盃決賽周參賽資格，也是韓國連續九次晉身世界盃決賽周。韓國的世界盃成績雖然是亞洲最佳，但在亞洲盃足球賽成績就遠不如世界盃。韓國除了在首兩屆亞洲杯奪冠外，但之後一直與亞洲盃錦標無緣，自1992年至2011年更連續六屆未能打入過亞洲盃決賽。2015年亞洲盃足球賽，韓國以五連勝一球不失的姿態，廿七年來首次打入亞洲盃決賽，對手是東道主澳洲。雖然韓國在分組初賽曾以1-0擊敗澳洲，但這場決賽韓國卻先失一球，最後在下半場補時階段扳平，令比賽進入加時階段，可惜澳洲最後在加時階段攻入致勝一球，最後韓國以1-2敗陣，只得亞軍。'

In [10]:
data = test_set[0]
output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
print(output)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-11.9815, -11.2627, -15.0886,  ..., -15.2958, -15.2962, -15.2981],
        [-12.7801, -13.3592, -14.8644,  ..., -15.3204, -15.3648, -15.3551],
        [-12.8654, -13.3881, -15.3689,  ..., -15.4968, -15.5326, -15.5321],
        ...,
        [-12.4551, -11.5566, -14.5512,  ..., -14.7235, -14.9215, -14.9404],
        [-11.8784, -11.2368, -14.3930,  ..., -14.7891, -15.0371, -15.1306],
        [-11.7980, -11.9603, -14.4653,  ..., -14.0655, -14.4302, -14.6539]],
       device='cuda:0', grad_fn=<CloneBackward0>), end_logits=tensor([[-12.5659, -12.8300, -14.2616,  ..., -14.2505, -14.1121, -14.2462],
        [-14.0157, -15.0074, -15.1917,  ..., -15.0789, -15.0030, -14.9334],
        [-14.2028, -15.0088, -15.2453,  ..., -15.0352, -14.9889, -14.9522],
        ...,
        [-12.8251, -14.3696, -14.3306,  ..., -13.3479, -13.1698, -13.1673],
        [-12.2557, -14.2416, -14.3133,  ..., -13.0337, -12.8120, -12.6937],
        [-11.6966, -14

In [11]:
start_prob, start_index = torch.max(output.start_logits[0], dim=0)
end_prob, end_index = torch.max(output.end_logits[0], dim=0)
print(data[0][0][start_index : end_index + 1])

print(start_index)
print(end_index)

question_len = len(test_questions_tokenized[0]) + 2
print(question_len)
start_index -= question_len
end_index -= question_len
tokenized_paragraph = test_paragraphs_tokenized[test_questions[0]['paragraph_id']]
start_index = min(tokenized_paragraph.token_to_chars(start_index))
end_index = max(tokenized_paragraph.token_to_chars(end_index))
print(start_index)
print(end_index)
print(test_paragraphs[test_questions[0]['paragraph_id']][start_index:end_index+10])
print(test_paragraphs[test_questions[0]['paragraph_id']][start_index:end_index])

tensor([4166, 5760, 1162, 1046])
tensor(131, device='cuda:0')
tensor(134, device='cuda:0')
58
76
80
烏茲別克，以小組次名取得20
烏茲別克


In [12]:
print(test_paragraphs_tokenized.token_to_chars(start_index))
print(test_paragraphs_tokenized.token_to_chars(end_index))
test_paragraphs_tokenized.fromkeys


CharSpan(start=76, end=77)
CharSpan(start=80, end=81)


<bound method UserDict.fromkeys of <class 'transformers.tokenization_utils_base.BatchEncoding'>>

In [13]:
test_paragraphs[test_questions[0]['paragraph_id']][133:137]

'，三戰一'

In [14]:
from tqdm.auto import tqdm
result = []

model.eval()
with torch.no_grad():
    idx = 0
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, tokenizer, idx))
        idx += 1

result_file = "result.csv"
with open(result_file, 'w') as f:	
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

  3%|▎         | 113/3524 [00:06<02:58, 19.08it/s]

empty~ num~ unlls


  6%|▌         | 206/3524 [00:11<02:47, 19.86it/s]

empty~ num~ unlls


  7%|▋         | 247/3524 [00:13<03:37, 15.07it/s]

...

empty~ num~ unlls


 97%|█████████▋| 3426/3524 [03:12<00:05, 18.38it/s]

empty~ num~ unlls


100%|██████████| 3524/3524 [03:17<00:00, 17.85it/s]

Completed! Result is in result.csv
